In [3]:
from get_stock_list import stock_list
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import seaborn
import matplotlib.pyplot as plt
%matplotlib notebook
from IPython.display import display, HTML

In [1]:
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import pandas as pd

start = dt.datetime.now() - dt.timedelta(days=30)
now = dt.datetime.now()
df = pdr.get_data_yahoo('F', start, now)
# df['Index'] = pd.to_datetime(df['Index']).dt.date

print(df.dtypes)
# for row in df.itertuples():
#     print(row.Index.date())
#     break
df.index.values
df['Volume'].tolist()

High         float64
Low          float64
Open         float64
Close        float64
Volume         int64
Adj Close    float64
dtype: object


[76930200,
 93039200,
 61879700,
 80046000,
 98876600,
 115798600,
 208648200,
 113169300,
 115749600,
 147327100,
 116184700,
 88719000,
 104742600,
 109535200,
 67732200,
 61301800,
 126127800,
 71724300,
 86672800,
 75536300,
 73612700,
 118316100]

In [6]:
import pandas as pd
sp_500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].filter(['Symbol', 'Security'], axis=1)
stock_stats = {}
for row in sp_500.itertuples():
    stock_stats[row[1]] = {}
    stock_stats[row[1]]['Name'] = row[2]

active_df = pd.read_html('https://finance.yahoo.com/most-active')[0]
active_df['Symbol'].to_list()
active_df = active_df.set_index('Symbol')

records = active_df.to_dict(orient='records')
records

# for _, record in active_df.iterrows():
#     for  i, value in enumerate(record):
#         print(i,value)
        
#     break
    
# active_df.to_dict()

# active_df.iloc[:, '%Change']

[{'Name': 'General Electric Company',
  'Price (Intraday)': 7.24,
  'Change': -0.23,
  '% Change': '-3.08%',
  'Volume': '68.996M',
  'Avg Vol (3 month)': '121M',
  'Market Cap': '63.329B',
  'PE Ratio (TTM)': nan,
  '52 Week Range': nan},
 {'Name': 'American Airlines Group Inc.',
  'Price (Intraday)': 16.98,
  'Change': -0.05,
  '% Change': '-0.29%',
  'Volume': '71.707M',
  'Avg Vol (3 month)': '91.747M',
  'Market Cap': '7.181B',
  'PE Ratio (TTM)': nan,
  '52 Week Range': nan},
 {'Name': 'Norwegian Cruise Line Holdings Ltd.',
  'Price (Intraday)': 19.2,
  'Change': -1.76,
  '% Change': '-8.40%',
  'Volume': '66.268M',
  'Avg Vol (3 month)': '57.755M',
  'Market Cap': '4.922B',
  'PE Ratio (TTM)': nan,
  '52 Week Range': nan},
 {'Name': 'Ford Motor Company',
  'Price (Intraday)': 6.33,
  'Change': -0.22,
  '% Change': '-3.36%',
  'Volume': '64.356M',
  'Avg Vol (3 month)': '104.085M',
  'Market Cap': '25.175B',
  'PE Ratio (TTM)': nan,
  '52 Week Range': nan},
 {'Name': 'NIO Inc.',


In [16]:
import pandas as pd
table=pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')

# df.to_csv('S&P500-Info.csv')
# df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])
df = table[1]
# for row in df.itertuples():
#     print(row[0], row[1], row[2])
#     break

dj = ''
for row in df.itertuples():
#     print(row[3])
    if ':' in row[3]:
      dj = dj + row[3].split(':')[1].strip() + ','
    else:
        dj = dj +row[3] + ','
dj

'MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DOW,XOM,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,RTX,TRV,UNH,VZ,V,WMT,WBA,DIS,'

In [18]:
def get_most_active_stock_stats(url_most_active):
    most_active_stats_df = pd.read_html(url_most_active)[0]
    most_active = most_active_stats_df.to_dict('records')   
    most_active_stats = {}
    for row in most_active:
        most_active_stats[row['Symbol']] = row
    most_active_stats_df = pd.DataFrame(most_active_stats).T
    
    return most_active_stats, most_active_stats_df

def get_stock_stats(stock_tickers, url_prefix, stock_stats):
    for sym in stock_tickers:
        stock_url = url_prefix.format(sym)
        df_list = pd.read_html(stock_url)
        if sym not in stock_stats.keys():
            stock_stats[sym] = {}
        for df in df_list:
            for item in df.to_dict('records'):
                stock_stats[sym][item[0]] = item[1]
    stock_stats_df = pd.DataFrame(stock_stats).T
    
    
    return stock_stats


def get_stock_data(stock_tickers, start, now):
    stock_dict = {}
    for stock in stock_tickers:
        print(f"Analysis for {stock}")
        df = pdr.get_data_yahoo(stock, start, now)
        for ma in mas:
            smaString = f"Sma_{ma}"
            df[smaString] = df.iloc[:, 4].rolling(window=ma).mean()
        df['day_percent_change'] =  round((df['Adj Close'] -  df['Open'])*100/df['Open'],2)
        for ma in mas:
            smaString = f"Sma_{ma}"
            df[f'{smaString}_percent_change'] =  round((df['Close'] -  df[smaString])*100/df[smaString],2)
        
        df = df.sort_values(by='Date', ascending=False)

        stock_dict[stock] = df
    return stock_dict

def plot_group_grphs(stock_groups, stock_dict, plt_col):
#     print("ploting graphs for stocks")
    for ticker_group in stock_groups.keys():
        fig, axes = plt.subplots()
        for ticker in stock_groups[ticker_group]:
            df = pd.DataFrame()
            for col in plt_col:
                df[ticker+'_'+col] = stock_dict[ticker][col]
                df[ticker+'_'+col].plot(legend = 'l', title = ticker_group + ' moving avrerages')
        
#         stock_stats, stock_stats_df = get_stock_stats(stock_groups[ticker_group], url_prefix)
#         display(HTML(stock_stats_df.to_html()))




mas = [20, 50, 200]
# for i in range(1, 17):
#     mas.append(90*i)
url_prefix = 'https://sg.finance.yahoo.com/quote/{0}/key-statistics?p={0}'
url_most_active = 'https://finance.yahoo.com/most-active'
url_summary = 'https://finance.yahoo.com/quote/{0}?p={0}'
url_financial = 'https://finance.yahoo.com/quote/{0}/financials?p={0}'
plt_col = ['Adj Close']
stock_stats = {}
yf.pdr_override()


start = dt.datetime(2019, 1, 1)
now = dt.datetime.now()


most_active_stats, most_active_stats_df = get_most_active_stock_stats(url_most_active)

stock_tickers = ['AAPL']

# stock_stats = get_stock_stats(stock_tickers, url_prefix, stock_stats)
# stock_stats = get_stock_stats(stock_tickers, url_summary, stock_stats)
stock_dict = get_stock_data(stock_tickers, start, now)
# stock_stats
stock_dict
# stock_test = {}
# stock_test['most active'] = stock_tickers
# plot_group_grphs(stock_test, stock_dict, plt_col)

Analysis for AAPL
[*********************100%***********************]  1 of 1 completed


{'AAPL':                   Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2020-06-12  344.720001  347.799988  334.220001  338.799988  338.799988   
 2020-06-11  349.309998  351.059998  335.480011  335.899994  335.899994   
 2020-06-10  347.899994  354.769989  346.089996  352.839996  352.839996   
 2020-06-09  332.140015  345.609985  332.010010  343.989990  343.989990   
 2020-06-08  330.250000  333.600006  327.320007  333.459991  333.459991   
 2020-06-05  323.350006  331.750000  323.230011  331.500000  331.500000   
 2020-06-04  324.390015  325.619995  320.779999  322.320007  322.320007   
 2020-06-03  324.660004  326.200012  322.299988  325.119995  325.119995   
 2020-06-02  320.750000  323.440002  318.929993  323.339996  323.339996   
 2020-06-01  317.750000  322.350006  317.209991  321.850006  321.850006   
 2020-05-29  319.250000  321.149994  316.470001  317.940002  317.940002   
 2020-05-28  316.

In [25]:
stock_stats['GE']

my_stats = ['Forward P/E 1', '52-week high 3', '52-week low 3', '50-day moving average 3',
           '200-day moving average 3', 'Beta (5Y Monthly)','PE Ratio (TTM)','EPS (TTM)', 'Open']

my_stock_dict = {}
for key in stock_stats.keys():
    my_stock_dict[key] ={}
    for key2 in my_stats:
        my_stock_dict[key][key2] = float(stock_stats[key][key2])


In [29]:
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

def get_page(url):
    # Set up the request headers that we're going to use, to simulate
    # a request by the Chrome browser. Simulating a request from a browser
    # is generally good practice when building a scraper
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Cache-Control': 'max-age=0',
        'Pragma': 'no-cache',
        'Referrer': 'https://google.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
    }

    return requests.get(url, headers=headers)

def parse_rows(table_rows):
    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)
            
    return pd.DataFrame(parsed_rows)

def clean_data(df):
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names
    
    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    
    numeric_columns = list(df.columns)[1::] # Take all columns, except the first (which is the 'Date' column)

    for column_index in range(1, len(df.columns)): # Take all columns, except the first (which is the 'Date' column)
        df.iloc[:,column_index] = df.iloc[:,column_index].str.replace(',', '') # Remove the thousands separator
        df.iloc[:,column_index] = df.iloc[:,column_index].astype(np.float64) # Convert the column to float64
        
    return df

def scrape_table(url):
    # Fetch the page that we're going to parse
    page = get_page(url);

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Fetch all div elements which have class 'D(tbr)'
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")
    
    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0
    
    df = parse_rows(table_rows)
    df = clean_data(df)
        
    return df

def get_statement(symbol):
    statements = {}
    statements['balance'] = scrape_table(f'https://finance.yahoo.com/quote/{symbol}/balance-sheet/?p={symbol}')
    statements['income'] = scrape_table(f'https://finance.yahoo.com/quote/{symbol}/financials/?p={symbol}')
    statements['cash'] = scrape_table(f'https://finance.yahoo.com/quote/{symbol}/cash-flow/?p={symbol}')
    
    return statements
symbol = 'AAPL'
df_balance_sheet = scrape_table('https://finance.yahoo.com/quote/' + symbol + '/balance-sheet?p=' + symbol)
get_statement('AAPL')['cash']

,Date,Operating Cash Flow,Investing Cash Flow,Financing Cash Flow,End Cash Position,Income Tax Paid Supplemental Data,Interest Paid Supplemental Data,Capital Expenditure,Issuance of Capital Stock,Issuance of Debt,Repayment of Debt,Repurchase of Capital Stock,Free Cash Flow
1,ttm,75373000.0,22049000.0,-94190000.0,43049000.0,13271000.0,3350000.0,-8737000.0,821000.0,13247000.0,-11555000.0,-73679000.0,66636000.0
2,9/30/2019,69391000.0,45896000.0,-90976000.0,50224000.0,15263000.0,3423000.0,-10495000.0,781000.0,6963000.0,-8805000.0,-66897000.0,58896000.0
3,9/30/2018,77434000.0,16066000.0,-87876000.0,25913000.0,10417000.0,3022000.0,-13313000.0,669000.0,6969000.0,-6500000.0,-72738000.0,64121000.0
4,9/30/2017,63598000.0,-46446000.0,-17347000.0,20289000.0,11591000.0,2092000.0,-12795000.0,555000.0,28662000.0,-3500000.0,-32900000.0,50803000.0
5,9/30/2016,65824000.0,-45977000.0,-20483000.0,20484000.0,10444000.0,1316000.0,-13548000.0,495000.0,24954000.0,-2500000.0,-29722000.0,52276000.0
